In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
path='/content/drive/MyDrive/Search_Engine/Copy of eng_subtitles_database.db'

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
# Read the code below and write your observation in the next cell

conn = sqlite3.connect(path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

In [ ]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

In [ ]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

In [ ]:
df.info()
df.shape

In [ ]:
b_data = df.iloc[0, 2]

# here 2 represent the index of content column
# 0 represents the row number

In [ ]:
df_sample = df.sample(frac=0.11, random_state=42)
df_sample.head()
df_sample.shape

In [ ]:
print(b_data)

In [ ]:
import zipfile
import io

# Assuming 'content' is the binary data from your database
binary_data = df.iloc[385, 2]

# Decompress the binary data using the zipfile module
with io.BytesIO(binary_data) as f:
    with zipfile.ZipFile(f, 'r') as zip_file:
        # Reading only one file in the ZIP archive
        subtitle_content = zip_file.read(zip_file.namelist()[0])

# Now 'subtitle_content' should contain the extracted subtitle content
print(subtitle_content.decode('latin-1'))  # Assuming the content is latin-1 encoded text

In [ ]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [ ]:
df_sample['file_content'] = df_sample['content'].apply(decode_method)

df_sample.head()

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
import re

def clean_text(text):
    # Remove BOM characters (if present)
    text = re.sub(r'ï»¿', '', text)

    # Pattern explanation:
    # (?:\d+\s+)?             --> Optionally matches a numeric subtitle index with trailing whitespace.
    # \d{2}:\d{2}:\d{2},\d{3}   --> Matches a timestamp in the format HH:MM:SS,mmm.
    # \s+-->\s+                --> Matches spaces, the literal arrow (-->) and additional spaces.
    # \d{2}:\d{2}:\d{2},\d{3}   --> Matches the ending timestamp.
    pattern = r'(?:\d+\s+)?\d{2}:\d{2}:\d{2},\d{3}\s+-->\s+\d{2}:\d{2}:\d{2},\d{3}'

    # Remove any timestamp pattern found anywhere in the text.
    text = re.sub(pattern, '', text)

    # Remove the <i> and </i> tags (both opening and closing tags).
    text = re.sub(r'</?i>', '', text)

    # Normalize consecutive whitespace into a single space and convert the text to lowercase.
    text = re.sub(r'\s+', ' ', text)

    return text.strip().lower()

# Apply the updated cleaning function to your DataFrame column.
df_sample['clean_subtitle'] = df_sample['file_content'].apply(clean_text)


In [ ]:
df_sample['clean_subtitle']

In [18]:
def chunk_text(text, max_tokens=500, overlap=50):
    tokens = text.split()
    chunks = []
    start = 0
    while start < len(tokens):
        chunk = tokens[start: start + max_tokens]
        chunks.append(' '.join(chunk))
        start += (max_tokens - overlap)
    return chunks

df_sample['chunks'] = df_sample['clean_subtitle'].apply(chunk_text)


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Flatten all chunks for TF-IDF vectorization
all_docs = [chunk for chunks in df_sample['chunks'] for chunk in chunks]
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_docs)


In [20]:
all_docs

['watch any video online with open-subtitles free browser extension: osdb.link/ext it could\'ve been just another summer. but as i set foot on the sand, that summer suddenly felt different. like it was going to be the summer that would change my life. the summer of freedom. the summer of endless possibilities. the summer of 2007. ooh, aah! ooh, oh! â\x99ª oh, oh, ooh â\x99ª that was the summer of you and me. you\'re quite the dancer. why did you stop? come on! keep dancing! whatever! i\'m kidding. don\'t get mad. huh? what... hey! i\'m just going to get my towel. what? stop that! you thought i was gonna kiss you. no! excuse me! i wanna kiss you but not just yet. what do you mean "not yet"? only when you\'re my girl. - what do you mean your girl? - my girlfriend, miss. as if! you wish! and don\'t call me miss. don\'t pretend to be a gentleman when you\'re clearly not. so, what should i call you? rude? snob? bitch? and you? douche? handsome. - conceited. - just like you. - huh? jerk! - e

In [21]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 24187979 stored elements and shape (105321, 196364)>

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [23]:
# Install TensorFlow and TensorFlow Hub if needed
!pip install tensorflow tensorflow_hub --quiet


In [24]:
!pip install langchain chromadb sentence-transformers --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.6 MB/s eta 0:00

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the Universal Sentence Encoder model from TF Hub.
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
print("Universal Sentence Encoder loaded successfully.")

# Define a function to compute embeddings for a list of text chunks.





In [26]:
def compute_embeddings_for_chunks(chunk_list):
    """
    Accepts a list of text (chunks) and returns the embeddings as a NumPy array.
    """
    # Compute embeddings (this returns a TensorFlow tensor)
    embeddings = embed(chunk_list)
    # Convert tensor to a Numpy array
    embeddings_np = embeddings.numpy()
    return embeddings_np

# Apply the function to the DataFrame column 'chuncks'
# This creates a new column 'embeddings' where each row contains the embeddings for that row's chunks.
df_sample['embeddings'] = df_sample['chunks'].apply(compute_embeddings_for_chunks)

print("Embeddings computed successfully for each row.")

Embeddings computed successfully for each row.


In [27]:
first_row_embeddings = df_sample['embeddings'].iloc[0]
similarity_matrix = cosine_similarity(first_row_embeddings)
print("Cosine similarity matrix for the first row's chunks:")
print(similarity_matrix)

Cosine similarity matrix for the first row's chunks:
[[1.0000002  0.6634338  0.62690866 0.67464006 0.5829463  0.5872241
  0.4608324  0.59522974 0.6057955  0.585786   0.61255014 0.5155581
  0.54593736 0.5448955  0.59197974 0.5667813  0.6009442  0.54781246
  0.4790931  0.5914738  0.5311309  0.48722613 0.5302949  0.63685447
  0.34839916]
 [0.6634338  1.0000002  0.60934275 0.5967097  0.43416283 0.50221425
  0.37520957 0.4929285  0.50610197 0.536508   0.5819875  0.5209473
  0.47770235 0.40333182 0.44278425 0.48136252 0.5549307  0.53799444
  0.4891033  0.46363723 0.41919035 0.47568205 0.37241575 0.59300053
  0.35565746]
 [0.62690866 0.60934275 0.9999998  0.720302   0.58965766 0.59776306
  0.49020743 0.56248367 0.5844308  0.6167634  0.6179102  0.615958
  0.6322638  0.57438046 0.59555733 0.66791856 0.6303193  0.6022519
  0.5138989  0.5996146  0.49348524 0.5294014  0.57486    0.61768556
  0.35995978]
 [0.67464006 0.5967097  0.720302   0.99999976 0.60431224 0.5870483
  0.4703993  0.53142834 0.59

In [28]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9075 entries, 17262 to 64473
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   num             9075 non-null   int64 
 1   name            9075 non-null   object
 2   content         9075 non-null   object
 3   file_content    9075 non-null   object
 4   clean_subtitle  9075 non-null   object
 5   chunks          9075 non-null   object
 6   embeddings      9075 non-null   object
dtypes: int64(1), object(6)
memory usage: 567.2+ KB


In [29]:
df_sample.head()

,num,name,content,file_content,clean_subtitle,chunks,embeddings
17262,9251120,maybe.this.time.(2014).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x89\x9a\x...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",watch any video online with open-subtitles fre...,[watch any video online with open-subtitles fr...,"[[-0.039998803, -0.060925838, 0.030114144, 0.0..."
7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,b'PK\x03\x04\x14\x00\x00\x00\x08\x007\x8f\x99V...,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ...","¶ oh, i know that it's getting late ¶ ¶ but i ...","[¶ oh, i know that it's getting late ¶ ¶ but i...","[[-0.0071158125, -0.036883183, 0.02758421, 0.0..."
47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x8f\x19\x...,"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin...",timing and subtitles by the uncontrollable lov...,[timing and subtitles by the uncontrollable lo...,"[[-0.015693832, -0.06250157, 0.064639665, 0.00..."
29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00[\xaa\x99V...,"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea...",[ethereal music] api.opensubtitles.org is depr...,[[ethereal music] api.opensubtitles.org is dep...,"[[-0.02531635, 0.0019707815, 0.060364593, 0.00..."
54266,9408707,battlebots.(2015).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xf4<\x9aV...,"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri...","[chris] oh, no, not the minibots! [yelling] oh...","[[chris] oh, no, not the minibots! [yelling] o...","[[-0.054557454, -0.05962945, 0.031208117, 0.01..."


In [30]:
# Save the DataFrame to a CSV file in your Google Drive
# output_path = '/content/drive/MyDrive/df_manipulated.pkl'
# df_sample.to_pickle(output_path)
# print("DataFrame saved to:", output_path)


In [31]:
!pip install chromadb --quiet

In [32]:
df_sample.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9075 entries, 17262 to 64473
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   num             9075 non-null   int64 
 1   name            9075 non-null   object
 2   content         9075 non-null   object
 3   file_content    9075 non-null   object
 4   clean_subtitle  9075 non-null   object
 5   chunks          9075 non-null   object
 6   embeddings      9075 non-null   object
dtypes: int64(1), object(6)
memory usage: 567.2+ KB


In [33]:
import pandas as pd
df_sample=pd.DataFrame(df_sample)

In [34]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9075 entries, 17262 to 64473
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   num             9075 non-null   int64 
 1   name            9075 non-null   object
 2   content         9075 non-null   object
 3   file_content    9075 non-null   object
 4   clean_subtitle  9075 non-null   object
 5   chunks          9075 non-null   object
 6   embeddings      9075 non-null   object
dtypes: int64(1), object(6)
memory usage: 567.2+ KB


In [35]:
import chromadb

# Initialize the ChromaDB client
client = chromadb.Client()

# Delete the collection named "subtitle_embeddings" if it exists.
client.delete_collection("subtitle_embeddings")
print("Collection 'subtitle_embeddings' deleted successfully.")


ValueError: Collection subtitle_embeddings does not exist.

In [ ]:
!pip install pandas
!pip install chromadb

In [ ]:
df_sample.info()

In [ ]:
import chromadb

if hasattr(chromadb, "_SYSTEM"):
    del chromadb._SYSTEM
    print("Existing ChromaDB client instance deleted successfully.")

In [43]:
import chromadb
import pandas as pd
from chromadb.config import Settings

# Initialize ChromaDB client and create/get the collection
client = chromadb.Client()
collection = client.get_or_create_collection(name="subtitle_embeddings")

# Ensure df_sample is loaded and structured correctly
if "chunks" in df_sample.columns and "embeddings" in df_sample.columns:
    for idx, row in df_sample.iterrows():
        try:
            # Validate chunks and embeddings
            if not isinstance(row["chunks"], list) or not isinstance(row["embeddings"], list):
                print(f"Skipping row {idx} due to invalid chunks or embeddings format.")
                continue

            for chunk_idx, (chunk_text, embedding) in enumerate(zip(row["chunks"], row["embeddings"])):
                # Create a unique ID, e.g., combining row index and chunk index
                doc_id = f"{idx}_{chunk_idx}"

                # Add document, embedding, and ID to the collection
                collection.add(
                    documents=[chunk_text],
                    embeddings=[embedding],
                    ids=[doc_id]
                )
        except Exception as e:
            print(f"Error processing row {idx}, chunk {chunk_idx}: {e}")

    print("Data has been successfully added to the ChromaDB collection.")
else:
    print("Error: The DataFrame does not contain 'chunks' or 'embeddings' columns.")


UniqueConstraintError: Collection subtitle_embeddings already exists

In [44]:
# Install necessary libraries in Google Colab
!pip install gradio
!pip install sentence-transformers
!pip install openai-whisper
!pip install chromadb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 22.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803403 sha256=e946727bc9fa710ef0cf47245e9bb58e6456bb583a7675439bd2f21a95cbc14b
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper


In [45]:
# Import required libraries
import gradio as gr
#from sentence_transformers import SentenceTransformer
import whisper


In [46]:
# -------------------------------
# Setup models and Chroma DB client
# -------------------------------

# Load Whisper model for audio transcription.
model_whisper = whisper.load_model("base")

# Load embedding model (same model as used for document embeddings).
#embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize Chroma DB client.
import chromadb
subtitle_embeddings = chromadb.Client()

# Ensure you configure your Chroma DB instance and load your embeddings here.


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 57.9MiB/s]


In [57]:
subtitle_embeddings

In [58]:
# Get all document IDs and count manually
all_documents = collection.get()["ids"]
doc_count = len(all_documents)
print(f"Total documents in the collection: {doc_count}")


NotImplementedError: In Chroma v0.6.0, list_collections only returns collection names. Use Client.get_collection(subtitle_embeddings) to access get. See https://docs.trychroma.com/deployment/migration for more information.

In [47]:
import tensorflow_hub as hub  # Ensure TensorFlow Hub is installed

def transcribe_audio(audio_file: str) -> str:
    """
    Takes an audio file and transcribes it to text using Whisper.
    """
    audio = whisper.load_audio(audio_file)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model_whisper.device)
    options = whisper.DecodingOptions()
    result = whisper.decode(model_whisper, mel, options)
    return result.text

def retrieve_documents_from_query(audio_file: str) -> str:
    """
    Given an audio file input (recorded via Gradio), transcribes the audio,
    creates an embedding for the transcribed text, and uses Chroma DB to retrieve
    the most relevant documents based on cosine similarity.
    """
    # Step 1: Transcribe audio query to text
    query_text = transcribe_audio(audio_file)

    # Step 2: Preprocess and create query embedding
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    query_embedding = embed([query_text])[0]  # Ensure embedding is generated as a vector

    # Step 3: Query the collection for similar documents
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5  # Number of top documents to retrieve
    )

    # Step 4: Prepare output
    output = f"Transcribed Query:\n{query_text}\n\nTop Matching Documents:\n"
    for doc, score in zip(results["documents"], results["distances"]):
        output += f"Document: {doc}\nSimilarity Score: {score:.4f}\n\n"

    return output


In [49]:
iface = gr.Interface(
    fn=retrieve_documents_from_query,
    inputs=gr.Audio(type="filepath"),  # Removed `source="microphone"`
    outputs="text",
    title="Audio Query-based Document Retrieval",
    description=(
        "Record a 2-minute audio excerpt from a TV series or movie. "
        "This interface will transcribe your query, convert it into an embedding, "
        "and retrieve the most relevant documents from your Chroma DB."
    )
)

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9ef2f92fce928d5c4d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [51]:
# Assuming you have a collection named "subtitles"
collection = subtitle_embeddings.get_collection("subtitles")

# List all document IDs in the collection
document_ids = collection.get()["ids"]
print("Document IDs in Collection:")
print(document_ids)

# Check the count of documents
print(f"Total documents in collection: {len(document_ids)}")


InvalidCollectionException: Collection subtitles does not exist.

In [42]:
# def retrieve_documents_from_query(audio_file: str) -> str:
#     """
#     Given an audio file input (recorded via Gradio), this function transcribes
#     the audio, creates an embedding for the transcribed text, compares it against
#     stored document embeddings, computes cosine similarity scores, and returns
#     the top relevant documents.
#     """
#     # Step 1: Transcribe the audio query to text.
#     query_text = transcribe_audio(audio_file)

#     # (Optional) Preprocess the text if required (e.g., lowercasing, punctuation removal).
#     # For now, we assume the raw transcription is sufficient.

#     # Step 2: Create query embedding.
#     embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
#     query_embedding = embed.encode(query_text)

#     # Step 3: For each stored document compute cosine similarity.
#     # Using cosine similarity formula: dot(query, doc) / (||query|| * ||doc||)
#     results = subtitle_embeddings.query(
#         query_embeddings=[query_embedding],
#         n_results=5  # Number of top documents to retrieve.
#     )

#     # Step 4: Sort documents by similarity score (highest first).
#     results = sorted(results, key=lambda x: x[1], reverse=True)

#     # Step 5: Return top candidate documents (for example, top 3)
#     top_docs = results[:6]

#     # Create an output string that shows the transcription and the matching documents.
#     output = f"Transcribed Query:\n{query_text}\n\nTop Matching Documents:\n"
#     for title, score, content in top_docs:
#         output += f"Title: {title}\nScore: {score:.4f}\nContent: {content}\n\n"

#     return output

In [ ]:
query_clean = clean_text(query_text)


In [ ]:
query_embedding = model.encode(query_clean)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Transform the query into a TF-IDF vector
query_vector = tfidf_vectorizer.transform([query_clean])
# Compute cosine similarities: returns an array of similarity scores
cosine_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
# Retrieve indices of top matches
top_indices = np.argsort(cosine_scores)[::-1][:5]

print("Top relevant documents (keyword-based):")
for i in top_indices:
    print(all_docs[i], "\nScore:", cosine_scores[i])


In [ ]:
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)

# Print out the top document chunks
print("Top relevant documents (semantic-based):")
for doc in results['documents'][0]:
    print(doc)
